In [2]:
from __future__ import print_function, division
import scipy

from keras.datasets import mnist
from keras_contrib.layers.normalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
from data_loader import DataLoader
import numpy as np
import os



## sigin
import signal
import sys

Using TensorFlow backend.


In [3]:
## source from keras-gan

class colorizeSketch():
    def __init__(self):
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        self.dataset_name = 'anime'
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))


        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminators
        self.d_A = self.build_discriminator()
        self.d_B = self.build_discriminator()
        self.d_A.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d_B.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators
        self.g_AB = self.build_generator()
        self.g_BA = self.build_generator()

        # Input images from both domains
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Translate images to the other domain
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        # Translate images back to original domain
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)

        # For the combined model we will only train the generators
        self.d_A.trainable = False
        self.d_B.trainable = False

        # Discriminators determines validity of translated images
        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)

        # Objectives
        # + Adversarial: Fool domain discriminators
        # + Translation: Minimize MAE between e.g. fake B and true B
        # + Cycle-consistency: Minimize MAE between reconstructed images and original
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[ valid_A, valid_B,
                                        fake_B, fake_A,
                                        reconstr_A, reconstr_B ])
        self.combined.compile(loss=['mse', 'mse',
                                    'mae', 'mae',
                                    'mae', 'mae'],
                              optimizer=optimizer)
        

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4, normalize=True):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalize:
                d = InstanceNormalization()(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = InstanceNormalization()(u)
            u = Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf, normalize=False)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)
        d5 = conv2d(d4, self.gf*8)
        d6 = conv2d(d5, self.gf*8)
        d7 = conv2d(d6, self.gf*8)

        # Upsampling
        u1 = deconv2d(d7, d6, self.gf*8)
        u2 = deconv2d(u1, d5, self.gf*8)
        u3 = deconv2d(u2, d4, self.gf*8)
        u4 = deconv2d(u3, d3, self.gf*4)
        u5 = deconv2d(u4, d2, self.gf*2)
        u6 = deconv2d(u5, d1, self.gf)

        u7 = UpSampling2D(size=2)(u6)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1,
                            padding='same', activation='tanh')(u7)

        return Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, normalization=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalization:
                d = InstanceNormalization()(d)
            return d

        img = Input(shape=self.img_shape)

        d1 = d_layer(img, self.df, normalization=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50,load=False):

        start_time = datetime.datetime.now()


        if load == True:
            1+1
        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)
        
        
        Gloss=100
        for epoch in range(epochs):

            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):

                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to opposite domain
                fake_B = self.g_AB.predict(imgs_A)
                fake_A = self.g_BA.predict(imgs_B)

                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # Total disciminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)

                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B], [valid, valid, \
                                                                         imgs_B, imgs_A, \
                                                                         imgs_A, imgs_B])

                elapsed_time = datetime.datetime.now() - start_time
                # Plot the progress
                print ("[%d] [%d/%d] time: %s, [d_loss: %f, g_loss: %f]" % (epoch, batch_i,
                                                                        self.data_loader.n_batches,
                                                                        elapsed_time,
                                                                        d_loss[0], g_loss[0]))

                
                # If at save interval => save generated image samples
                if batch_i % 4 == 0 and epoch % 10 == 0  :
                    self.sample_images(epoch, batch_i,g_loss[0])
                if g_loss[0] < Gloss and epoch > 10 :
                    self.g_BA.save('./models/gen_model2.h5')
                    self.g_BA.save_weights('./models/gen_weights2.h5')
                    self.g_AB.save('./models/gen_model.h5')
                    self.g_AB.save_weights('./models/gen_weights.h5')
                    self.combined.save('./models/model.h5')
                    self.combined.save_weights('./models/weights.h5')
                    Gloss = g_loss[0] 
                ## Save for case of perf no inc 
                if epoch % 9==0 and batch_i == 20 :
                    self.g_BA.save_weights('./models/gen_weights2-s.h5')
                    self.g_AB.save_weights('./models/gen_weights-s.h5')

    def sample_images(self, epoch, batch_i,g_loss):
        os.makedirs('images/%s' % self.dataset_name, exist_ok=True)
        r, c = 2, 3

        imgs_A, imgs_B = self.data_loader.load_data(batch_size=1, is_testing=True)

        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(fake_A)

        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated:'+str(round(g_loss,4))+"%", 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
        plt.close()

def signal_handler(self,sig, frame):
        print('Graceful Interupt')


In [ ]:

 ## dis learning false

gan = colorizeSketch()
    
gan.train(epochs=500, batch_size=3, sample_interval=100,load=False)


C:\Users\me\Anaconda3\envs\tf-gpu\lib\site-packages\keras\engine\training.py:478: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[0] [0/833] time: 0:01:18.084480, [d_loss: 14.017591, g_loss: 163.491425]
[0] [1/833] time: 0:01:19.858749, [d_loss: 46.627769, g_loss: 45.022568]
[0] [2/833] time: 0:01:21.055252, [d_loss: 31.693771, g_loss: 4.554713]
[0] [3/833] time: 0:01:22.252012, [d_loss: 3.043244, g_loss: 2.366658]
[0] [4/833] time: 0:01:23.446599, [d_loss: 0.404094, g_loss: 3.158381]
[0] [5/833] time: 0:01:24.977013, [d_loss: 0.628204, g_loss: 2.686956]
[0] [6/833] time: 0:01:26.169511, [d_loss: 0.478254, g_loss: 2.266435]
[0] [7/833] time: 0:01:27.367739, [d_loss: 0.380102, g_loss: 2.333819]
[0] [8/833] time: 0:01:28.584883, [d_loss: 0.303749, g_loss: 2.548820]
[0] [9/833] time: 0:01:30.107262, [d_loss: 0.309173, g_loss: 2.782613]
[0] [10/833] time: 0:01:31.300193, [d_loss: 0.290680, g_loss: 3.290475]
[0] [11/833] time: 0:01:32.487996, [d_loss: 0.523864, g_loss: 2.507844]
[0] [12/833] time: 0:01:33.682892, [d_loss: 0.399367, g_loss: 2.330888]
[0] [13/833] time: 0:01:35.229528, [d_loss: 0.379064, g_loss: 2.6541

[0] [114/833] time: 0:03:59.617019, [d_loss: 0.629633, g_loss: 1.500622]
[0] [115/833] time: 0:04:00.809432, [d_loss: 0.400576, g_loss: 1.809661]
[0] [116/833] time: 0:04:01.999764, [d_loss: 0.446557, g_loss: 1.861980]
[0] [117/833] time: 0:04:03.520358, [d_loss: 0.414501, g_loss: 1.914236]
[0] [118/833] time: 0:04:04.723435, [d_loss: 0.392145, g_loss: 1.833913]
[0] [119/833] time: 0:04:05.915859, [d_loss: 0.507063, g_loss: 1.617587]
[0] [120/833] time: 0:04:07.115271, [d_loss: 0.311296, g_loss: 2.312974]
[0] [121/833] time: 0:04:08.663992, [d_loss: 0.529512, g_loss: 1.509876]
[0] [122/833] time: 0:04:09.862846, [d_loss: 0.372382, g_loss: 1.997113]
[0] [123/833] time: 0:04:11.054356, [d_loss: 0.486145, g_loss: 1.720336]
[0] [124/833] time: 0:04:12.249690, [d_loss: 0.414231, g_loss: 1.770111]
[0] [125/833] time: 0:04:13.810935, [d_loss: 0.317219, g_loss: 2.261483]
[0] [126/833] time: 0:04:15.018004, [d_loss: 0.474901, g_loss: 1.602004]
[0] [127/833] time: 0:04:16.216969, [d_loss: 0.3847

[0] [227/833] time: 0:06:25.227402, [d_loss: 0.415605, g_loss: 1.343869]
[0] [228/833] time: 0:06:26.427648, [d_loss: 0.338306, g_loss: 1.586298]
[0] [229/833] time: 0:06:27.969810, [d_loss: 0.426393, g_loss: 1.479166]
[0] [230/833] time: 0:06:29.166805, [d_loss: 0.325638, g_loss: 1.588388]
[0] [231/833] time: 0:06:30.363829, [d_loss: 0.412729, g_loss: 1.563758]
[0] [232/833] time: 0:06:31.559563, [d_loss: 0.399012, g_loss: 1.595174]
[0] [233/833] time: 0:06:33.099648, [d_loss: 0.387541, g_loss: 1.574882]
[0] [234/833] time: 0:06:34.312417, [d_loss: 0.373733, g_loss: 1.420444]
[0] [235/833] time: 0:06:35.504808, [d_loss: 0.350720, g_loss: 1.454261]
[0] [236/833] time: 0:06:36.704724, [d_loss: 0.403122, g_loss: 1.461325]
[0] [237/833] time: 0:06:38.237927, [d_loss: 0.307831, g_loss: 1.777778]
[0] [238/833] time: 0:06:39.436666, [d_loss: 0.358823, g_loss: 1.663650]
[0] [239/833] time: 0:06:40.644532, [d_loss: 0.414391, g_loss: 1.367184]
[0] [240/833] time: 0:06:41.840311, [d_loss: 0.3747

[0] [340/833] time: 0:08:49.949377, [d_loss: 0.373589, g_loss: 1.143297]
[0] [341/833] time: 0:08:51.475821, [d_loss: 0.313800, g_loss: 1.441887]
[0] [342/833] time: 0:08:52.670532, [d_loss: 0.297413, g_loss: 1.544665]
[0] [343/833] time: 0:08:53.870170, [d_loss: 0.336225, g_loss: 1.345377]
[0] [344/833] time: 0:08:55.068832, [d_loss: 0.349303, g_loss: 1.222188]
[0] [345/833] time: 0:08:56.626976, [d_loss: 0.339566, g_loss: 1.337391]
[0] [346/833] time: 0:08:57.816131, [d_loss: 0.332963, g_loss: 1.373058]
[0] [347/833] time: 0:08:59.019755, [d_loss: 0.354230, g_loss: 1.339558]
[0] [348/833] time: 0:09:00.219122, [d_loss: 0.329235, g_loss: 1.400393]
[0] [349/833] time: 0:09:01.776692, [d_loss: 0.326828, g_loss: 1.302531]
[0] [350/833] time: 0:09:02.973986, [d_loss: 0.357464, g_loss: 1.158275]
[0] [351/833] time: 0:09:04.172536, [d_loss: 0.343033, g_loss: 1.273996]
[0] [352/833] time: 0:09:05.374677, [d_loss: 0.339661, g_loss: 1.230855]
[0] [353/833] time: 0:09:06.912939, [d_loss: 0.3278

...

[5] [479/833] time: 1:35:14.026735, [d_loss: 0.265267, g_loss: 1.355798]
[5] [480/833] time: 1:35:15.247558, [d_loss: 0.263102, g_loss: 1.133171]
[5] [481/833] time: 1:35:16.515354, [d_loss: 0.264070, g_loss: 1.126427]
[5] [482/833] time: 1:35:17.736626, [d_loss: 0.239360, g_loss: 1.028879]
[5] [483/833] time: 1:35:18.946227, [d_loss: 0.219288, g_loss: 1.136823]
[5] [484/833] time: 1:35:20.161332, [d_loss: 0.224138, g_loss: 1.125628]
[5] [485/833] time: 1:35:21.371866, [d_loss: 0.173242, g_loss: 1.695449]
[5] [486/833] time: 1:35:22.565476, [d_loss: 0.143950, g_loss: 1.424692]
[5] [487/833] time: 1:35:23.770980, [d_loss: 0.156073, g_loss: 1.540664]
[5] [488/833] time: 1:35:24.972202, [d_loss: 0.136900, g_loss: 1.678260]
[5] [489/833] time: 1:35:26.176195, [d_loss: 0.164720, g_loss: 1.581687]
[5] [490/833] time: 1:35:27.381074, [d_loss: 0.166330, g_loss: 1.867247]
[5] [491/833] time: 1:35:28.580494, [d_loss: 0.242600, g_loss: 1.735340]
[5] [492/833] time: 1:35:29.779912, [d_loss: 0.2348

[5] [592/833] time: 1:37:31.720320, [d_loss: 0.183186, g_loss: 1.485716]
[5] [593/833] time: 1:37:32.945489, [d_loss: 0.149734, g_loss: 1.583767]
[5] [594/833] time: 1:37:34.169286, [d_loss: 0.215195, g_loss: 1.499377]
[5] [595/833] time: 1:37:35.391551, [d_loss: 0.173010, g_loss: 1.547115]
[5] [596/833] time: 1:37:36.611358, [d_loss: 0.125358, g_loss: 1.793745]
[5] [597/833] time: 1:37:37.832866, [d_loss: 0.239402, g_loss: 1.209021]
[5] [598/833] time: 1:37:39.058333, [d_loss: 0.157544, g_loss: 1.384660]
[5] [599/833] time: 1:37:40.281779, [d_loss: 0.143944, g_loss: 1.767753]
[5] [600/833] time: 1:37:41.497799, [d_loss: 0.331978, g_loss: 1.020680]
[5] [601/833] time: 1:37:42.715859, [d_loss: 0.283464, g_loss: 1.392750]
[5] [602/833] time: 1:37:43.946090, [d_loss: 0.281788, g_loss: 1.171133]
[5] [603/833] time: 1:37:45.177747, [d_loss: 0.265748, g_loss: 1.210052]
[5] [604/833] time: 1:37:46.403683, [d_loss: 0.267475, g_loss: 1.324194]
[5] [605/833] time: 1:37:47.631255, [d_loss: 0.2357

[5] [705/833] time: 1:39:49.849839, [d_loss: 0.260498, g_loss: 1.067500]
[5] [706/833] time: 1:39:51.072859, [d_loss: 0.271168, g_loss: 0.941397]
[5] [707/833] time: 1:39:52.296334, [d_loss: 0.259928, g_loss: 0.910783]
[5] [708/833] time: 1:39:53.523491, [d_loss: 0.257318, g_loss: 0.976887]
[5] [709/833] time: 1:39:54.743419, [d_loss: 0.265101, g_loss: 0.961627]
[5] [710/833] time: 1:39:55.959740, [d_loss: 0.273138, g_loss: 0.973937]
[5] [711/833] time: 1:39:57.183298, [d_loss: 0.269964, g_loss: 0.865925]
[5] [712/833] time: 1:39:58.399857, [d_loss: 0.253680, g_loss: 0.993469]
[5] [713/833] time: 1:39:59.620243, [d_loss: 0.267192, g_loss: 0.921499]
[5] [714/833] time: 1:40:00.843741, [d_loss: 0.270908, g_loss: 0.887758]
[5] [715/833] time: 1:40:02.062140, [d_loss: 0.274608, g_loss: 0.847199]
[5] [716/833] time: 1:40:03.283306, [d_loss: 0.270883, g_loss: 0.989080]
[5] [717/833] time: 1:40:04.503429, [d_loss: 0.238773, g_loss: 1.084377]
[5] [718/833] time: 1:40:05.728936, [d_loss: 0.2745

[5] [818/833] time: 1:42:08.300786, [d_loss: 0.255835, g_loss: 0.916551]
[5] [819/833] time: 1:42:09.543541, [d_loss: 0.284963, g_loss: 1.108926]
[5] [820/833] time: 1:42:10.746557, [d_loss: 0.267671, g_loss: 0.929179]
[5] [821/833] time: 1:42:11.949301, [d_loss: 0.264642, g_loss: 0.917748]
[5] [822/833] time: 1:42:13.155012, [d_loss: 0.258232, g_loss: 1.060462]
[5] [823/833] time: 1:42:14.352539, [d_loss: 0.269213, g_loss: 0.922084]
[5] [824/833] time: 1:42:15.548009, [d_loss: 0.277084, g_loss: 0.954269]
[5] [825/833] time: 1:42:16.765249, [d_loss: 0.259924, g_loss: 1.258086]
[5] [826/833] time: 1:42:18.026970, [d_loss: 0.270203, g_loss: 0.896365]
[5] [827/833] time: 1:42:19.255490, [d_loss: 0.266456, g_loss: 0.922260]
[5] [828/833] time: 1:42:20.497373, [d_loss: 0.263453, g_loss: 0.958408]
[5] [829/833] time: 1:42:21.714532, [d_loss: 0.260708, g_loss: 1.007819]
[5] [830/833] time: 1:42:22.954188, [d_loss: 0.258511, g_loss: 0.986608]
[5] [831/833] time: 1:42:24.204699, [d_loss: 0.2681

[6] [100/833] time: 1:44:28.663121, [d_loss: 0.239714, g_loss: 1.196966]
[6] [101/833] time: 1:44:29.916280, [d_loss: 0.228681, g_loss: 1.185679]
[6] [102/833] time: 1:44:31.169436, [d_loss: 0.272954, g_loss: 1.051839]
[6] [103/833] time: 1:44:32.410092, [d_loss: 0.264834, g_loss: 0.923846]
[6] [104/833] time: 1:44:33.654873, [d_loss: 0.265422, g_loss: 0.991147]
[6] [105/833] time: 1:44:34.892983, [d_loss: 0.259279, g_loss: 1.081601]
[6] [106/833] time: 1:44:36.123441, [d_loss: 0.269635, g_loss: 0.961203]
[6] [107/833] time: 1:44:37.348577, [d_loss: 0.252245, g_loss: 1.026480]
[6] [108/833] time: 1:44:38.561312, [d_loss: 0.268659, g_loss: 0.945261]
[6] [109/833] time: 1:44:39.756933, [d_loss: 0.232468, g_loss: 1.290187]
[6] [110/833] time: 1:44:40.994128, [d_loss: 0.275912, g_loss: 1.285064]
[6] [111/833] time: 1:44:42.208294, [d_loss: 0.256066, g_loss: 1.241653]
[6] [112/833] time: 1:44:43.413545, [d_loss: 0.285902, g_loss: 0.975263]
[6] [113/833] time: 1:44:44.628359, [d_loss: 0.2799